In [1]:
import pandas as pd
import numpy as np
import os
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx
from tqdm import tqdm
from utils import *
from tdc.multi_pred import DTI

In [2]:
data = DTI(name = 'BindingDB_Ki')
data.convert_to_log(form = 'binding')

Found local copy...
Loading...
Done!
To log space...


In [3]:
data = data.harmonize_affinities(mode = 'max_affinity')

In [31]:
from sklearn.model_selection import train_test_split
train_ic50, test_ic50 = train_test_split(data, test_size=0.2)

In [3]:
split = data.get_split(method = 'cold_split', column_name = 'Drug')
test_ic50 = split['test']
train_ic50 = split['train']

Valid set

In [4]:
valid_ic50 = split['valid']

In [5]:
valid_ic50 = valid_ic50.drop(columns=['Drug_ID', 'Target_ID'])

In [6]:
valid_ic50.reset_index(drop=True, inplace=True)

end of valid set

In [7]:
test_ic50 = test_ic50.drop(columns=['Drug_ID', 'Target_ID'])
train_ic50 = train_ic50.drop(columns=['Drug_ID', 'Target_ID'])

In [30]:
train_ic50.reset_index(drop=True, inplace=True)
test_ic50.reset_index(drop=True, inplace=True)

In [9]:
frames = [train_ic50, valid_ic50]
train_ic50 = pd.concat(frames)

In [42]:
train_ic50

,Drug,Target,Y
0,O=C1N(C/C=C/c2cn[nH]c2)[C@H](Cc2ccccc2)[C@H](O...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,9.292430
1,O=C1N(CCCCCCO)[C@H](Cc2ccccc2)[C@H](O)[C@@H](O...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,9.045757
2,O=C1N(CCCCCO)[C@H](Cc2ccccc2)[C@H](O)[C@@H](O)...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,8.962574
3,CCCCN1C(=O)N(CC2CC2)[C@H](Cc2ccccc2)[C@H](O)[C...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,8.920819
4,O=C1N(CC2CCC2)[C@H](Cc2ccccc2)[C@H](O)[C@@H](O...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,8.853872
...,...,...,...
300243,O=C(O)CCCCc1nc2cc(C3=NOC4(C3)Cc3ccccc3C4)ccc2c...,MSANATLKPLCPILEQMSRLQSHSNTSIRYIDHAAVLLHGLASLLG...,8.920819
300244,C[C@@]1(c2ccccc2)CC(c2ccc3c(=O)n(-c4ccc(F)cc4)...,MSANATLKPLCPILEQMSRLQSHSNTSIRYIDHAAVLLHGLASLLG...,8.420216
300245,C[C@]1(Cc2ccccc2)CC(c2ccc3c(=O)n(-c4ccc(F)cc4)...,MSANATLKPLCPILEQMSRLQSHSNTSIRYIDHAAVLLHGLASLLG...,8.124939
300246,O=C1CCCN1[C@H]1CC[C@H](C(=O)Nc2ccc(-c3cc(F)cc(...,MDLELDEYYNKTLATENNTAATRNSDFPVWDDYKSSVDDLQYFLIG...,6.142607


In [34]:
train_ic50.to_csv('../data/bdtdc_ki_train.csv', index = False)
test_ic50.to_csv('../data/bdtdc_ki_test.csv', index = False)

In [12]:
list4 = test_ic50[test_ic50['Drug'].astype(str).str.len() <10 ].index
list4 = set(list4)

In [32]:
str1 = ['[Br-].[Na+]','[I-].[Na+]']
index_drop = []
for st1 in str1:
    index_drop.extend(test_ic50[test_ic50['Drug'].astype(str) == st1].index)

index_drop = set(index_drop)
index_drop

set()

In [19]:
len(list4)

283

In [29]:
train_ic50 = train_ic50.drop(index_drop)


In [25]:
train_ic50

,Drug,Target,Y
0,CC(=O)N[C@H](C(=O)N[C@@H](Cc1ccccc1)[C@@H](O)C...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,8.065502
1,COC(=O)N[C@H](C(=O)NN(CC1CCCCC1)C[C@H](O)[C@H]...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.567031
2,CCOC(=O)N[C@H](C(=O)NN(CC1CCCCC1)C[C@H](O)[C@H...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.251037
3,CC(C)[C@@H]1[C@H](OC(=O)N[C@@H](Cc2ccccc2)[C@H...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.317855
4,CC(C)[C@@H]1[C@H](OC(=O)N[C@@H](Cc2ccccc2)[C@H...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,7.459671
...,...,...,...
198713,CCN(CCCCOc1ccc(-c2coc3cc(O)cc(O)c3c2=O)cc1)Cc1...,MKILDALLFPVIFIMFFIHLSIAQTDPELTIMTRLGQVQGTRLPVP...,4.798600
198714,CC(C)NCc1cccc2[nH]c3cc(O)ccc3c12,EEDIIITTKNGKVRGMNLPVLGGTVTAFLGIPYAQPPLGRLRFKKP...,4.762705
198715,CC(C)NCc1cccc2[nH]c3cc(O)ccc3c12,MKILDALLFPVIFIMFFIHLSIAQTDPELTIMTRLGQVQGTRLPVP...,4.361610
198716,CC(C)NCc1cccc2oc3cc(O)ccc3c12,EEDIIITTKNGKVRGMNLPVLGGTVTAFLGIPYAQPPLGRLRFKKP...,4.736125


In [33]:
stat2 = train_ic50.groupby(['Drug', 'Target']).Y.agg(["mean", "median", "var"]).reset_index()
stat3 = test_ic50.groupby(['Drug', 'Target']).Y.agg(["mean", "median", "var"]).reset_index()


In [41]:
stat2.loc[stat2['var'] > 0]

,Drug,Target,mean,median,var
5,Br.CC1(C)C2CC1[C@]1(C)OB(C(CCC/C(S)=N/N)NC(=O)...,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,9.641597,9.972650,3.499654e-01
18,Br.C[S+](CCC(N)C#N)C[C@H]1O[C@@H](n2cnc3c(N)nc...,MEAAHFFEGTEKLLEVWFSRQQPDANQGSGDLRTIPRSEWDILLKD...,4.057622,4.057622,4.043086e-01
26,Br.c1ccc2c(c1)CCN(CCCc1nc3ccccc3s1)C2,MASLSQLSSHLNYTCGAENSTGASQARPHAYYALSYCALILAIVFG...,7.099096,7.099096,1.488415e-05
27,Br.c1ccc2c(c1)CCN(CCCc1nc3ccccc3s1)C2,MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWT...,5.688968,5.688968,1.823724e-06
31,Br.c1ccc2c(c1)CCN(CCCc1nc3ccccc3s1)C2,MSLPNSSCLLEDKMCEGNKTTMASPQLMPLVVVLSTICLVTVGLNL...,6.087466,6.087466,1.142748e-05
...,...,...,...,...,...
267661,c1n[nH]c2c1CCN1CCCCC21,MVFLSGNASDSSNCTHPPPPVNISKAILLGVILGGLILFGVLGNIL...,4.619896,4.619896,2.358806e-08
267708,c1ncc(C2CCCc3sccc32)[nH]1,MGSLQPDAGNSSWNGTEAPGGGTRATPYSLQVTLTLVCLAGLLMLF...,8.373743,8.373743,1.301571e+00
267725,c1ncc(CC2CCNCC2)[nH]1,MERAPPDGLMNASGTLAGEAAAAGGARGFSAAWTAVLAALMALLIV...,9.022879,9.022879,7.917813e-02
267726,c1ncc(CC2CCNCC2)[nH]1,MERAPPDGPLNASGALAGEAAAAGGARGFSAAWTAVLAALMALLIV...,8.497439,9.258059,2.984526e+00


In [34]:
train_ic50_cleaned = stat2.drop(stat2.loc[stat2['var'] > 1].index)
train_ic50_cleaned = train_ic50_cleaned.drop(columns = ['mean', 'var'])
train_ic50_cleaned = train_ic50_cleaned.rename(columns = {'median':'Y'})

In [35]:
test_ic50_cleaned = stat3.drop(stat3.loc[stat3['var'] > 1].index)
test_ic50_cleaned = test_ic50_cleaned.drop(columns = ['mean', 'var'])
test_ic50_cleaned = test_ic50_cleaned.rename(columns = {'median':'Y'})

In [36]:
train_ic50_cleaned.reset_index(drop=True, inplace=True)
test_ic50_cleaned.reset_index(drop=True, inplace=True)

In [27]:
stats4 = test_ic50_cleaned.groupby(['Drug', 'Target']).size().reset_index(name='counts')
stats4 = stats4.sort_values(by=['counts'],ascending=False)
stats4

,Drug,Target,counts
0,B[P@@](=O)(OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C...,MERDGCAGGGSRGGEGGRAPREGPAGNGRDRGRSHAAEAPGDPQAA...,1
43889,Cc1nc2c(=S)n(Cc3ccccc3)nc(-c3ccncc3)c2c2cc(-c3...,MPIMGSSVYITVELAIAVLAILGNVLVCWAVWLNSNLQNVTNYFVV...,1
43923,Cc1nc2cc(C(=O)N[C@@H]3C4CCN(CC4)C3(C)C)ccc2s1,MRCSPGGVWLALAASLLHVSLQGEFQRKLYKELVKNYNPLERPVAN...,1
43924,Cc1nc2cc(C(=O)N[C@@H]3C4CCN(CC4)C34CC4)ccc2o1,MLLWVQQALLALLLPTLLAQGEARRSRNTTRPALLRLSDYLLTNYR...,1
43925,Cc1nc2cc(C(=O)N[C@@H]3C4CCN(CC4)C34CC4)ccc2o1,MRCSPGGVWLALAASLLHVSLQGEFQRKLYKELVKNYNPLERPVAN...,1
...,...,...,...
21971,CN1C(=O)c2c(nn(Cc3ccc(C(F)(F)F)nc3)c2Nc2ccccc2...,MGSGSSSYRPKAIYLDIDGRIQKVIFSKYCNSSDIMDLFCIATGLP...,1
21972,CN1C(=O)c2c(nn(Cc3ccc(C(F)(F)F)nc3)c2Nc2ccccc2...,MRIEERKSQHLTGLTDEKVKAYLSLHPQVLDEFVSESVSAETVEKW...,1
21973,CN1C(=O)c2c(nn(Cc3ccc(C(F)(F)F)nc3)c2Nc2ccccc2...,MRRDERDAKAMRSLQPPDGAGSPPESLRNGYVKSCVSPLRQDPPRG...,1
21974,CN1C(=O)c2c(nn(Cc3ccc(C(F)(F)F)nc3)c2Nc2ccccc2...,MSCLMVERCGEILFENPDQNAKCVCMLGDIRLRGQTGVRAERRGSY...,1


In [10]:
stats2.loc[stats2['counts'] > 1]

,Drug,Target,counts
410513,Cc1ccsc1C(=CCCN1CCC[C@@H](C(=O)O)C1)c1sccc1C,MATDNSKVADGQISTEVSEAPVASDKPKTLVVKVQKKAGDLPDRDT...,2
511991,O=C(CC1(c2ccc(F)cc2)C2CC3CC1CC(C2)C3O)N1CC(O)C1,MAFMKKYLLPILGLFMAYYYYSANEEFRPEMLQGKKVIVTGASKGI...,2
493749,N[C@H]1CCN(c2c(F)cc3c(=O)c(C(=O)O)cn4c3c2Oc2cc...,MEVSPLQPVNENMQVNKIKKNEDAKKRLSVERIYQKKTQLEHILLR...,2
552238,O=C(O)CCc1ncc[nH]1,MTAEQALPLGNGKAAEEARGSETLGGGGGGAAGTREARDKAVHERG...,2
41181,CC(C)(C)OC(=O)Nc1ccc2c(c1)N(S(=O)(=O)c1cccc(C(...,MDRAPQRQHRASRELLAAKKTHTSQIEVIPCKICGDKSSGIHYGVI...,2
...,...,...,...
555608,O=C(O)[C@@H]1CCCN(CCOCCC(c2ccccc2)c2ccccc2)C1,MATDNSKVADGQISTEVSEAPVASDKPKTLVVKVQKKAGDLPDRDT...,2
280698,COc1ccc(C(OCCN2CCC[C@H](C(=O)O)C2)(c2ccc(OC)cc...,MATDNSKVADGQISTEVSEAPVASDKPKTLVVKVQKKAGDLPDRDT...,2
497504,Nc1ccc2c(c1)Oc1c(N3CCC(N4CCCCC4)CC3)c(F)cc3c(=...,MEVSPLQPVNENMQVNKIKKNEDAKKRLSVERIYQKKTQLEHILLR...,2
484406,NCCCC(=O)O,MATDNSKVADGQISTEVSEAPVASDKPKTLVVKVQKKAGDLPDRDT...,2


In [39]:
train_ic50_cleaned

,Drug,Target,Y
0,B[P@@](=O)(OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C...,MERDGCAGGGSRGGEGGRAPREGPAGNGRDRGRSHAAEAPGDPQAA...,6.300943
1,B[P@](=O)(OC[C@H]1O[C@@H](n2cnc3c(N)nc(SC)nc32...,MERDGCAGGGSRGGEGGRAPREGPAGNGRDRGRSHAAEAPGDPQAA...,4.251811
2,B[P@](=O)(OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C@...,MERDGCAGGGSRGGEGGRAPREGPAGNGRDRGRSHAAEAPGDPQAA...,4.552840
3,B[P@](=O)(OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[C@...,MERDGCAGGGSRGGEGGRAPREGPAGNGRDRGRSHAAEAPGDPQAA...,5.154896
4,Br.CC1(C)C2CC1[C@]1(C)OB(C(CCC/C(S)=N/N)NC(=O)...,CGVPAIQPVLSGLSRIVNGEEAVPGSWPWQVSLQDKTGFHFCGGSL...,9.023192
...,...,...,...
264986,c1ncc([C@H]2C[C@H]2CCNCC2CCCCC2)[nH]1,MERAPPDGPLNASGALAGEAAAAGGARGFSAAWTAVLAALMALLIV...,7.686133
264987,c1ncc([C@H]2C[C@H]2CCNCC2CCCCC2)[nH]1,MPDTNSTINLSLSTRVTLAFFMSLVAFAIMLGNALVILAFVVDKNL...,6.751781
264988,c1ncc([C@H]2C[C@H]2CNCC2CCCCC2)[nH]1,MERAPPDGPLNASGALAGEAAAAGGARGFSAAWTAVLAALMALLIV...,5.999957
264989,c1ncc([C@H]2C[C@H]2CNCC2CCCCC2)[nH]1,MPDTNSTINLSLSTRVTLAFFMSLVAFAIMLGNALVILAFVVDKNL...,5.999957


In [43]:
train_ic50_cleaned.to_csv('../data/bdtdc_ki_train.csv', index = False)
test_ic50_cleaned.to_csv('../data/bdtdc_ki_test.csv', index = False)

In [40]:
train_ic50_cleaned[~train_ic50_cleaned['Y'].astype(str).str.contains('0|1|2|3|4|5|6|7|8|9')]

,Drug,Target,Y


In [ ]:
# Strange smiles 
# Converting SMILES to graph: 173000/174658
# Smile [Ni+2]
# Target [12. 21. 11. 19. 11.  7.  5. 18. 22. 15. 21. 11. 21.  7. 17. 17.  6. 11.